In [2]:
import nltk
import pandas as pd
import ast
import re
import numpy as np

import string 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

import unidecode
import nltk.corpus
from gensim.models import Word2Vec

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import config

[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>


In [3]:
df = pd.read_csv("Dataset/Cleaned_Indian_Food_Dataset.csv")

In [4]:
df.rename(columns = {'Cleaned-Ingredients':'ingredients'}, inplace = True)
for i in range(len(df["ingredients"])):
    df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
    df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
    df["ingredients"][i] = df["ingredients"][i].split(',')

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_39124/1364071601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_39124/1364071601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_39124/1364071601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [5]:
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       pass
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    remove_punctuations = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for each_item in ingredients:
        each_item.translate(remove_punctuations)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', each_item)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list

In [6]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

In [7]:
df['parsed'] = df.ingredients.apply(ingredient_parser)
# get corpus
#corpus = get_and_sort_corpus(df)

In [8]:
import gensim
import numpy as np

def get_recommendations(input):

# Example data
    recipe_names = df['TranslatedRecipeName']
    ingredients = df['parsed']

# Tokenize and preprocess ingredients
    ingredient_tokens = ingredients

# Train FastText embeddings
    model = gensim.models.FastText(ingredient_tokens, vector_size=100, window=5, min_count=1, sg=1)

# Create embeddings for a query ingredient list
    query_ingredients = "tomato, salt, chickpea lentils, green chilli, rice, mustard, bc belle bhat powder, dry red chilli"
    query_embedding = np.mean([model.wv[token] for token in query_ingredients.split(', ')], axis=0)

# Compute cosine similarities
    similarities = [cosine_similarity([query_embedding], [np.mean([model.wv[token] for token in ing], axis=0)])[0][0] for ing in ingredient_tokens]

# Recommend recipes based on similarities
    recommendations = [recipe_names[i] for i in np.argsort(similarities)[::-1]]
    score = sorted(similarities, reverse=True)
    print("Recommended Recipes:")
    for i in range(len(recommendations)):
        print(recommendations[i],score[i])
# Calculate Precision at K (P@K) for K=3
    K = 3   
    actual_recipes = ["Spaghetti Carbonara", "Chicken Alfredo"]  # Actual recipes relevant to the query

# Calculate P@K
    correct_recommendations = [recipe in recommendations[:K] for recipe in actual_recipes]
    precision_at_K = sum(correct_recommendations) / K

    print(f"Precision at {K}: {precision_at_K}")


In [9]:
input = "tomato, salt, chickpea lentils, green chilli, rice, mustard, bc belle bhat powder, dry red chilli, cashew peanuts"

In [10]:
get_recommendations(input)

Recommended Recipes:
 Veggie Rava Idli (Recipe In Hindi) 0.9953468
Urad Dal Vada Recipe (Non Deep Fried) 0.9952696
Green Moong Dal And Vegetable Idli Recipe 0.9951448
Carrot Pulao Recipe - Carrot Rice (Recipe) 0.9949771
Carrot And Capsicum Rice Recipe 0.9949293
Konkani Style Solkadhi Recipe - Kokum Drink 0.9947618
Keerai Vadai Recipe Made With Figaro Pure Olive Oil 0.9946263
Khatua Recipe (Bhojpuri Style Khichdi) 0.99456185
Sindhi Sai Bhaji Recipe - Sindhi Sai Bhaji Recipe 0.9944495
Cucumber Mor Kuzhambu Recipe (Cucumber Curry) 0.99444044
Desiccated Coconut Coriander Chutney Recipe 0.9943923
Raw Mango Dal Fry Recipe 0.99411327
Crunchy Carrot Peanut Raita Recipe - Gajar Moongphali Raita 0.9940895
Palak Pulao Recipe 0.9940452
Bihari Style Baingan Ka Chokha Recipe (Smoked & Spiced Eggplant) 0.9939436
Beetroot Greens Rice Recipe 0.9939358
Vegetable Rice Upma Recipe 0.9939191
 Tomato Chutney (Recipe In Hindi) 0.9938254
Kheel Paniyaram Recipe (Puffed Rice Paniyaram) 0.99379
Dill Leaves Toor 

In [11]:
import gensim
import numpy as np

# Example data
recipe_names = ["Spaghetti Carbonara", "Chicken Alfredo", "Mushroom Risotto"]
ingredients = ["pasta, eggs, bacon", "chicken, cream, pasta", "rice, mushrooms, broth"]

# Tokenize and preprocess ingredients
ingredient_tokens = ingredients

# Train FastText embeddings
model = gensim.models.FastText(ingredient_tokens, vector_size=100, window=5, min_count=1, sg=1)

# Create embeddings for a query ingredient list
query_ingredients = "chicken, mushrooms, garlic"
query_embedding = np.mean([model.wv[token] for token in query_ingredients.split(', ')], axis=0)

# Compute cosine similarities
similarities = [cosine_similarity([query_embedding], [np.mean([model.wv[token] for token in ing.split(', ')], axis=0)])[0][0] for ing in ingredient_tokens]

# Recommend recipes based on similarities
recommendations = [recipe_names[i] for i in np.argsort(similarities)[::-1]]

# Calculate Precision at K (P@K) for K=3
K = 3
actual_recipes = ["Spaghetti Carbonara", "Chicken Alfredo"]  # Actual recipes relevant to the query

# Calculate P@K
correct_recommendations = [recipe in recommendations[:K] for recipe in actual_recipes]
precision_at_K = sum(correct_recommendations) / K

print(f"Precision at {K}: {precision_at_K}")


Precision at 3: 0.6666666666666666


In [17]:
from sklearn.metrics import ndcg_score
K = 3
hit_rates = [1 if any(recipe in recommendations[:K] for recipe in actual_recipes) else 0 for actual_recipes in relevant_recipes_list]
hr_at_K = sum(hit_rates) / len(hit_rates)
print(f"Hit Rate at {K}: {hr_at_K:.4f}")

# Compute NDCG for your recommendations and relevant items
ndcg = ndcg_score([relevant_recipes], [recommendations])
print(f"Normalized Discounted Cumulative Gain (NDCG): {ndcg:.4f}")

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Calculate precision and recall
precision, recall, _ = precision_recall_curve(binary_relevance, similarities)

# Plot the precision-recall curve
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


Hit Rate at 3: 1.0000


ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [14]:
import gensim
import numpy as np
from sklearn.metrics import average_precision_score

# Example data
recipe_names = ["Spaghetti Carbonara", "Chicken Alfredo", "Mushroom Risotto"]
ingredients = ["pasta, eggs, bacon", "chicken, cream, pasta", "rice, mushrooms, broth"]

# Tokenize and preprocess ingredients
ingredient_tokens = ingredients

# Train FastText embeddings
model = gensim.models.FastText(ingredient_tokens, vector_size=100, window=5, min_count=1, sg=1)

# Create embeddings for a list of query ingredient lists
query_ingredients_list = [
    "chicken, mushrooms, garlic",
    "pasta, bacon, eggs",
    "rice, broth, mushrooms",
]

# Corresponding relevant recipes for each query
relevant_recipes_list = [
    ["Chicken Alfredo"],
    ["Spaghetti Carbonara"],
    ["Mushroom Risotto"],
]

# Calculate MAP
average_precision_scores = []

for query_ingredients, relevant_recipes in zip(query_ingredients_list, relevant_recipes_list):
    query_embedding = np.mean([model.wv[token] for token in query_ingredients.split(', ')], axis=0)
    similarities = [cosine_similarity([query_embedding], [np.mean([model.wv[token] for token in ing.split(', ')], axis=0)])[0][0] for ing in ingredient_tokens]
    
    # Rank the recipes based on similarity
    ranked_recipes = [recipe_names[i] for i in np.argsort(similarities)[::-1]]
    
    # Create a binary relevance vector (1 if the recipe is relevant, 0 otherwise)
    binary_relevance = [1 if recipe in relevant_recipes else 0 for recipe in ranked_recipes]
    
    # Calculate the Average Precision (AP) for this query
    ap = average_precision_score(binary_relevance, np.arange(len(binary_relevance)) + 1)
    average_precision_scores.append(ap)

# Calculate Mean Average Precision (MAP)
map_score = np.mean(average_precision_scores)

print(f"Mean Average Precision (MAP): {map_score:.4f}")


Mean Average Precision (MAP): 0.3333


In [16]:
import gensim
import numpy as np

# Example data
recipe_names = ["Spaghetti Carbonara", "Chicken Alfredo", "Mushroom Risotto"]
ingredients = ["pasta","eggs", "bacon", "chicken", "cream", "pasta", "rice", "mushrooms", "broth"]

# Tokenize and preprocess ingredients
ingredient_tokens = ingredients

# Train FastText embeddings
model = gensim.models.FastText(ingredient_tokens, vector_size=100, window=5, min_count=1, sg=1)

# Create embeddings for a list of query ingredient lists
query_ingredients_list = [
    "chicken, mushrooms, garlic",
    "pasta, bacon, eggs",
    "rice, broth, mushrooms",
]

# Corresponding relevant recipes for each query
relevant_recipes_list = [
    ["Chicken Alfredo", "Mushroom Risotto"],
    ["Spaghetti Carbonara"],
    ["Mushroom Risotto", "Chicken Alfredo"],
]

# Calculate MRR
mrr_scores = []

for query_ingredients, relevant_recipes in zip(query_ingredients_list, relevant_recipes_list):
    query_embedding = np.mean([model.wv[token] for token in query_ingredients.split(', ')], axis=0)
    similarities = [cosine_similarity([query_embedding], [np.mean([model.wv[token] for token in ing.split(', ')], axis=0)])[0][0] for ing in ingredient_tokens]
    
    # Rank the recipes based on similarity
    ranked_recipes = [recipe_names[i] for i in np.argsort(similarities)[::-1]]
    
    # Find the reciprocal rank of the first relevant recipe
    for i, recipe in enumerate(ranked_recipes):
        if recipe in relevant_recipes:
            reciprocal_rank = 1 / (i + 1)
            mrr_scores.append(reciprocal_rank)
            break
    else:
        # If no relevant recipe is found, set reciprocal rank to 0
        mrr_scores.append(0)

# Calculate Mean Reciprocal Rank (MRR)
mrr = np.mean(mrr_scores)

print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")


IndexError: list index out of range